In [50]:
import requests
import json
import pandas as pd

In [31]:
def fetch_councillor(id_):
    url = "http://ws-old.parlament.ch/votes/councillors/" + str(id_)
    print(url)
    params = dict(
        format='json'
    )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
    }
    resp = requests.get(url=url, params=params, headers=headers)
    return resp

In [32]:
print(resp.json())

{'firstName': 'Thomas', 'updated': '2017-11-24T10:12:56Z', 'lastName': 'Ammann', 'affairVotes': [{'registrationNumber': 13756, 'meaningNo': 'Antrag Page (keine Abschreibung)', 'date': '2016-06-17T09:00:55Z', 'submissionText': '', 'id': 17496, 'meaningYes': 'Antrag der Kommission (Abschreibung der Initiative)', 'affairTitle': 'Umbauten in der Landwirtschaftszone. Kantonale Kompetenz', 'affairId': 20020453, 'councillorVote': {'decision': 'Yes', 'id': 3002217}, 'divisionText': None}, {'registrationNumber': 14243, 'meaningNo': 'Antrag der Minderheit Heim (Fristverlängerung)', 'date': '2016-09-30T08:15:43Z', 'submissionText': '', 'id': 17983, 'meaningYes': 'Antrag der Mehrheit (Abschreibung)', 'affairTitle': 'Krebsfrüherkennung', 'affairId': 20050464, 'councillorVote': {'decision': 'Yes', 'id': 3077015}, 'divisionText': None}, {'registrationNumber': 13469, 'meaningNo': 'Antrag der Minderheit Buillard (Eintreten)', 'date': '2016-06-07T12:21:01Z', 'submissionText': '', 'id': 17209, 'meaningYe

In [33]:
conseil = [
    "Doris Leuthard",
    "Ueli Maurer",
    "Simonetta Sommaruga",
    "Johann Schneider-Ammann",
    "Alain Berset",
    "Guy Parmelin",
    "Ignazio Cassis"
]

In [39]:
conseil_dict = {
    "Alain Berset" : 1161,
    "Ignazio Cassis" : 2664,
    "Doris Leuthard" : 2510,
    "Ueli Maurer" : 2310,
    "Johann Schneider-Ammann" : 2530,
    "Simonetta Sommaruga" : 2534,
    "Guy Parmelin" : 2621
}

In [40]:
votes = []
for k, v in conseil_dict.items():
    votes.append(fetch_councillor(v))
    

http://ws-old.parlament.ch/votes/councillors/1161
http://ws-old.parlament.ch/votes/councillors/2534
http://ws-old.parlament.ch/votes/councillors/2310
http://ws-old.parlament.ch/votes/councillors/2510
http://ws-old.parlament.ch/votes/councillors/2621
http://ws-old.parlament.ch/votes/councillors/2664
http://ws-old.parlament.ch/votes/councillors/2530


In [41]:
votes

[<Response [404]>,
 <Response [404]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [87]:
df = pd.DataFrame()
for v in votes:
    if v.status_code == requests.codes.ok:
        j = v.json()
        for vote in j['affairVotes']:  
            df.loc[vote['id'], j['id']] = vote['councillorVote']['decision']
        

In [88]:
df

,2310,2510,2621,2664,2530
1492,No,Yes,No,NaN,No
1503,NT,No,EH,NaN,NT
1504,No,No,No,NaN,NT
1505,No,Yes,No,NaN,NT
1506,No,Yes,No,NaN,NT
1507,EH,No,EH,NaN,NT
1508,No,Yes,No,NaN,NT
1509,No,Yes,No,NaN,NT
1510,No,Yes,No,NaN,NT
1511,No,Yes,No,NaN,NT


In [89]:
# EH = Abstention
# ES = Absent
# NT = Not participated
# P  = President